In [1]:
# Cell 1
!pip install tomotopy
!pip install nltk
import nltk
nltk.download('stopwords')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.2/17.2 MB 19.9 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
# Cell 2
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Cell 3
from pathlib import Path
project_folder = Path("/content/drive")/"MyDrive"/"562 Project"
articles = project_folder/"Raw Articles"/"no_linebreaks"


In [4]:
# Cell 4
text = []
for file in articles.glob("*"):
  with open(file,'r') as f:
    text.append(f.readline());


In [ ]:
words:list[str]
freq:list[int]
doc = "@#@".join(sum([[word]*fre for word,fre in zip(words,freq)]));

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
# Cell 5
import sys
import tomotopy as tp
import numpy as np

def hlda_example(corpus_name,documents, save_path):
    from nltk.stem.porter import PorterStemmer
    from nltk.corpus import stopwords
    try:
        cps = tp.utils.Corpus.load(corpus_name + '.cached.cps')
    except IOError:
        stemmer = PorterStemmer() #stemmer: normalizes words, like running -> run
        stops = set(stopwords.words('english'))
        cps = tp.utils.Corpus(
            tokenizer=tp.utils.SimpleTokenizer(stemmer=None),
            stopwords=lambda x: len(x) <= 2 or x in stops
        )
        cps.process(text)
        cps.save(corpus_name + '.cached.cps')

    np.random.seed(42)
    ridcs = np.random.permutation(len(cps))
    test_idcs = ridcs[:20]
    train_idcs = ridcs[20:]

    test_cps = cps[test_idcs]
    train_cps = cps[train_idcs]

    mdl = tp.HLDAModel(tw=tp.TermWeight.ONE, min_df=10, depth=4, rm_top=10, corpus=train_cps)
    mdl.train(0)
    print('Num docs:', len(mdl.docs), ', Vocab size:', len(mdl.used_vocabs), ', Num words:', mdl.num_words)
    print('Removed top words:', mdl.removed_top_words)
    print('Training...', file=sys.stderr, flush=True)
    for _ in range(0, 1000, 10):
        mdl.train(7)
        mdl.train(3, freeze_topics=True)
        print('Iteration: {:05}\tll per word: {:.5f}\tNum. of topics: {}'.format(mdl.global_step, mdl.ll_per_word, mdl.live_k))

    for _ in range(0, 100, 10):
        mdl.train(10, freeze_topics=True)
        print('Iteration: {:05}\tll per word: {:.5f}\tNum. of topics: {}'.format(mdl.global_step, mdl.ll_per_word, mdl.live_k))

    mdl.summary()
    print('Saving...', file=sys.stderr, flush=True)
    mdl.save(save_path, True)

    test_result_cps, ll = mdl.infer(test_cps)
    for doc in test_result_cps:
        print(doc.path, doc.get_words(top_n=10))

    return mdl

# You can get the sample data file 'enwiki-16000.txt'
# at https://drive.google.com/file/d/1OfyJ9TqaMiqzO6Qw-c_jXL-pmSIZf5Xt/view?usp=sharing

if __name__ == '__main__':
    mdl = hlda_example('accessibility',list(map(str,text)), 'test.hlda.tmm')


Training...


Num docs: 61 , Vocab size: 2404 , Num words: 186760
Removed top words: ['people', 'design', 'environment', 'user', 'users', 'route', 'accessibility', 'disability', 'navigation', 'research']
Iteration: 00010	ll per word: -7.91042	Num. of topics: 13
Iteration: 00020	ll per word: -7.64416	Num. of topics: 18
Iteration: 00030	ll per word: -7.50548	Num. of topics: 21
Iteration: 00040	ll per word: -7.43035	Num. of topics: 25
Iteration: 00050	ll per word: -7.38447	Num. of topics: 27
Iteration: 00060	ll per word: -7.36125	Num. of topics: 28
Iteration: 00070	ll per word: -7.36796	Num. of topics: 25
Iteration: 00080	ll per word: -7.32097	Num. of topics: 30
Iteration: 00090	ll per word: -7.31360	Num. of topics: 30
Iteration: 00100	ll per word: -7.28494	Num. of topics: 34
Iteration: 00110	ll per word: -7.26601	Num. of topics: 37
Iteration: 00120	ll per word: -7.26822	Num. of topics: 36
Iteration: 00130	ll per word: -7.25802	Num. of topics: 39
Iteration: 00140	ll per word: -7.26067	Num. of topics: 3

Saving...


Iteration: 01100	ll per word: -7.19565	Num. of topics: 44
<Basic Info>
| HLDAModel (current version: 0.12.7)
| 61 docs, 186760 words
| Total Vocabs: 36452, Used Vocabs: 2404
| Entropy of words: 7.25282
| Entropy of term-weighted words: 7.25282
| Removed Vocabs: people design environment user users route accessibility disability navigation research
|
<Training Info>
| Iterations: 1100, Burn-in steps: 0
| Optimization Interval: 10
| Log-likelihood per word: -7.19565
|
<Initial Parameters>
| tw: TermWeight.ONE
| min_cf: 0 (minimum collection frequency of words)
| min_df: 10 (minimum document frequency of words)
| rm_top: 10 (the number of top words to be removed)
| depth: 4 (the maximum depth level of hierarchy between 2 ~ 32767)
| alpha: [0.1] (hyperparameter of Dirichlet distribution for document-depth level, given as a single `float` in case of symmetric prior and as a list with length `depth` of `float` in case of asymmetric prior.)
| eta: 0.01 (hyperparameter of Dirichlet distributio

TF-IDF https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html<br/>

Doc2Vec https://radimrehurek.com/gensim/models/doc2vec.html<br/>


With stemmer = stemmer parameter:<br/>
Basic Info <br/>
| HLDAModel (current version: 0.12.7)<br/>
| 63 docs, 194213 words<br/>
| Total Vocabs: 36472, Used Vocabs: 2454<br/>
| Entropy of words: 7.26714<br/>
| Entropy of term-weighted words: 7.26714<br/>
| Removed Vocabs: people users design environment route user accessibility research navigation disabilities<br/>

Training Info:<br/>
| Iterations: 1100, Burn-in steps: 0<br/>
| Optimization Interval: 10<br/>
| Log-likelihood per word: -7.23255<br/>

Initial Parameters:<br/>
| tw: TermWeight.ONE<br/>
| min_cf: 0 (minimum collection frequency of words)<br/>
| min_df: 10 (minimum document frequency of words)<br/>
| rm_top: 10 (the number of top words to be removed)<br/>
| depth: 4 (the maximum depth level of hierarchy between 2 ~ 32767)<br/>
| alpha: [0.1] (hyperparameter of Dirichlet distribution for document-depth level, given as a single `float` in case of symmetric prior and as a list with length `depth` of `float` in case of asymmetric prior.)<br/>
| eta: 0.01 (hyperparameter of Dirichlet distribution for topic-word)<br/>
| gamma: 0.1 (concentration coeficient of Dirichlet Process)<br/>
| seed: 2993553996 (random seed)<br/>
| trained in version 0.12.7<br/>

Parameters:<br/>
| alpha (Dirichlet prior on the per-document depth level distributions)<br/>
|  [1.3133966  0.995187   0.28201452 0.486861  ]<br/>
| eta (Dirichlet prior on the per-topic word distribution)<br/>
|  0.01<br/>
| gamma (concentration coefficient of Dirichlet Process)<br/>
|  0.1<br/>
| Number of Topics: 39<br/>

Topics:<br/>
| #0 (63886, 63) : data information used using use<br/>
|   #8 (55774, 63) : https disability doi space also<br/>
|     #16 (6068, 25) : model path routes time cultural<br/>
|       #24 (7924, 23) : wheelchair mobility physical public spaces<br/>
|       #33 (1737, 2) : detection system visually impaired obstacle<br/>
|     #17 (874, 2) : blind urban city orientation citizens<br/>
|       #36 (663, 2) : contrast visual surfaces difference standards<br/>
|     #18 (1395, 2) : curb ramps pedestrian sidewalk ramp<br/>
|       #32 (1989, 2) : walking groups facilities malaysia bus<br/>
|     #19 (456, 2) : crossing intersection app signal vision<br/>
|       #35 (4658, 2) : impaired impairment apps paper visually<br/>
|     #20 (2508, 2) : way participant doi indoor mobile<br/>
|       #30 (1493, 2) : model physical features usability architectural<br/>
|     #21 (2584, 3) : health mental built covid-19 anxiety<br/>
|       #38 (3560, 3) : cognitive studies neighborhood older adults<br/>
|     #22 (3915, 2) : indoor planning ieee impaired blind<br/>
|       #34 (1276, 2) : building buildings evaluation work health<br/>
|     #23 (662, 3) : malaysia impaired visually persons learning<br/>
|       #42 (2276, 3) : built respondents per comments disability<br/>
|     #48 (902, 2) : system detection smart visually blind<br/>
|       #27 (2246, 2) : elevator building simulation elevators strategy<br/>
|     #49 (3952, 2) : system blind application visually impaired<br/>
|       #46 (1882, 2) : travel level activity model selection<br/>
|     #50 (1737, 2) : vol indoor impaired outdoor multimodal<br/>
|       #41 (2280, 2) : disabled travel stations density transit<br/>
|     #51 (1005, 2) : application blind bus mobile service<br/>
|       #28 (1115, 2) : transportation service autonomous focus public<br/>
|     #52 (1590, 2) : social inclusion disability level activities<br/>
|       #25 (1548, 2) : buildings building accessible main barriers<br/>
|     #53 (2044, 4) : blind doi may participants would<br/>
|       #64 (1365, 4) : transit transportation disability individuals service<br/>
|     #54 (1600, 2) : confidence segment criteria values sidewalk<br/>
|       #26 (1999, 2) : surfaces tactile feedback blind participants<br/>
|     #55 (1073, 2) : routing barriers barrier figure 2021<br/>
|       #39 (1275, 2) : transport street bus disability stations<br/>
|     #56 (759, 2) : contrast vision elements normal visibility<br/>
|       #29 (994, 2) : outcomes settings building criteria setting<br/>
|     #57 (578, 2) : database architectural disabled data nodes<br/>
|       #43 (571, 2) : path data building localization system<br/>
|

[ 0  8 23 42] [('transport', 0.02727966383099556), ('disabled', 0.016978252679109573), ('public', 0.015261350199580193), ('people', 0.014879817143082619), ('doi', 0.012590614147484303), ('dhaka', 0.010682945139706135), ('research', 0.007630675099790096), ('accessibility', 0.007439908571541309), ('disability', 0.0064860740676522255), ('barriers', 0.006295307073742151)]<br/>
[ 0  8 53 64] [('route', 0.026824865490198135), ('risk', 0.016765540465712547), ('barriers', 0.014960020780563354), ('mobility', 0.013928295113146305), ('wheelchair', 0.012122775427997112), ('routes', 0.0116069121286273), ('https', 0.011091049760580063), ('accessibility', 0.010575187392532825), ('org', 0.009543461725115776), ('doi', 0.00928553007543087)]<br/>
[ 0  8 57 43] [('design', 0.02287166379392147), ('none', 0.010959339328110218), ('students', 0.010482845827937126), ('needs', 0.009053367190063), ('patients', 0.008259212598204613), ('satisfied', 0.008259212598204613), ('entrance', 0.007623888086527586), ('built', 0.007306226063519716), ('educational', 0.006829733029007912), ('environment', 0.006670902017503977)]<br/>
[ 0  8 48 27] [('dataset', 0.01267605647444725), ('detection', 0.01232394389808178), ('pothole', 0.011267605237662792), ('visually', 0.007746479008346796), ('impaired', 0.007042253389954567), ('sov', 0.006690140813589096), ('potholes', 0.005985915660858154), ('system', 0.005985915660858154), ('vision', 0.005633802618831396), ('lite', 0.005633802618831396)]<br/>
[ 0  8 48 27] [('health', 0.02314932644367218), ('care', 0.016026455909013748), ('sci', 0.014245739206671715), ('people', 0.01170185673981905), ('prevention', 0.011447468772530556), ('shcs', 0.011447468772530556), ('south', 0.00915797520428896), ('africa', 0.008903587237000465), ('rehabilitation', 0.008649198338389397), ('cord', 0.008394810371100903)]<br/>
[ 0  8 16 31] [('user', 0.0181590486317873), ('smart', 0.016123982146382332), ('navigation', 0.010644959285855293), ('assistant', 0.010331871919333935), ('impaired', 0.0095491549000144), ('visually', 0.009236067533493042), ('cane', 0.008453349582850933), ('people', 0.008453349582850933), ('mode', 0.00782717578113079), ('system', 0.0075140888802707195)]<br/>
[ 0  8 57 43] [('2020', 0.02282157726585865), ('museum', 0.018672199919819832), ('museums', 0.015214384533464909), ('people', 0.015214384533464909), ('doi', 0.014177040196955204), ('pandemic', 0.013485477305948734), ('2021', 0.012793914414942265), ('https', 0.01244813296943903), ('disabled', 0.008644537068903446), ('org', 0.008644537068903446)]<br/>
[ 0  8 54 37] [('accessibility', 0.023970291018486023), ('edmonton', 0.011478730477392673), ('2018', 0.01080351136624813), ('people', 0.009115462191402912), ('urban', 0.008777853101491928), ('research', 0.0077650235034525394), ('standards', 0.0077650235034525394), ('https', 0.0077650235034525394), ('canada', 0.007427413947880268), ('coe', 0.006752194371074438)]<br/>
[ 0  8 53 64] [('tactile', 0.01924114301800728), ('bus', 0.0181622002273798), ('people', 0.0163639634847641), ('surfaces', 0.010789426043629646), ('design', 0.008631541393697262), ('stops', 0.008271893486380577), ('stop', 0.007912245579063892), ('visual', 0.0071929506957530975), ('universal', 0.006114008370786905), ('figure', 0.0055745369754731655)]<br/>
[ 0  8 16 24] [('disability', 0.017389917746186256), ('social', 0.015315890312194824), ('facilities', 0.011805999092757702), ('students', 0.011486917734146118), ('buildings', 0.01100829616189003), ('pwds', 0.010529674589633942), ('education', 0.009093809872865677), ('disabilities', 0.007817485369741917), ('university', 0.007657945156097412), ('built', 0.00749840447679162)]<br/>
[ 0  8 20 30] [('julian', 0.06896551698446274), ('dresden', 0.06896551698446274), ('jan', 0.06896551698446274), ('striegl', 0.06896551698446274), ('accessible', 0.03448275849223137), ('weber', 0.03448275849223137), ('informatik', 0.03448275849223137), ('fabian', 0.03448275849223137), ('ders', 0.03448275849223137), ('lu¨', 0.03448275849223137)]
[ 0  8 16 24] [('information', 0.011760243214666843), ('data', 0.009484066627919674), ('user', 0.00834597833454609), ('doi', 0.0075872535817325115), ('network', 0.007397572044283152), ('directions', 0.007018209435045719), ('wayfinding', 0.006828527897596359), ('vol', 0.006828527897596359), ('persons', 0.006259484216570854), ('navigation', 0.005690440069884062)]<br/>
[ 0  8 48 27] [('system', 0.025241902098059654), ('model', 0.0092553636059165), ('object', 0.008834665641188622), ('signs', 0.008413966745138168), ('proposed', 0.007572570350021124), ('test', 0.007572570350021124), ('participants', 0.007572570350021124), ('detection', 0.007151871919631958), ('easy', 0.007151871919631958), ('user', 0.0067311739549040794)]
[ 0  8 53 64] [('lane', 0.021570103242993355), ('detection', 0.014547279104590416), ('road', 0.012916980311274529), ('methods', 0.01266616489738226), ('segmentation', 0.009781790897250175), ('method', 0.009280160069465637), ('pedestrian', 0.00827690027654171), ('network', 0.00802608486264944), ('color', 0.007900677621364594), ('image', 0.007148231845349073)]<br/>
[ 0  8 16 24] [('2010', 0.017247987911105156), ('individuals', 0.015139900147914886), ('behaviours', 0.013031813316047192), ('2009', 0.011498658917844296), ('autism', 0.011307014152407646), ('challenge', 0.011115370318293571), ('architecture', 0.010348792187869549), ('mostafa', 0.00977385975420475), ('brand', 0.0095822149887681), ('rca', 0.009007282555103302)]<br/>
[ 0 68 68 68] [('learning', 0.018307767808437347), ('environment', 0.015338941477239132), ('universities', 0.013854527845978737), ('design', 0.011380504816770554), ('education', 0.011380504816770554), ('university', 0.011380504816770554), ('inclusion', 0.010390895418822765), ('universal', 0.009896091185510159), ('0-25', 0.009896091185510159), ('students', 0.009896091185510159)]<br/>
[ 0  8 53 64] [('road-traffic', 0.01535735372453928), ('mortality', 0.014766686595976353), ('cities', 0.013388462364673615), ('deaths', 0.012994684278964996), ('higher', 0.011616460047662258), ('urban', 0.010435124859213829), ('population', 0.010041346773505211), ('traffic', 0.009647568687796593), ('city', 0.009647568687796593), ('density', 0.008466233499348164)]<br/>
[ 0  8 16 33] [('impaired', 0.008439566008746624), ('visually', 0.008439566008746624), ('user', 0.007790368050336838), ('using', 0.0063739377073943615), ('data', 0.006314919795840979), ('system', 0.006196883972734213), ('learning', 0.005960811860859394), ('obstacle', 0.005960811860859394), ('sensors', 0.005606704391539097), ('used', 0.005488668568432331)]<br/>
[ 0  8 17 36] [('children', 0.032258063554763794), ('school', 0.019825268536806107), ('schools', 0.018145160749554634), ('inclusive', 0.01646505296230316), ('disability', 0.013776881620287895), ('special', 0.011424731463193893), ('ghana', 0.010416666977107525), ('education', 0.010080644860863686), ('needs', 0.010080644860863686), ('disabilities', 0.009072580374777317)]<br/>
[ 0  8 54 37] [('dataset', 0.025744864717125893), ('images', 0.013884871266782284), ('datasets', 0.010702922008931637), ('object', 0.01041365321725607), ('detection', 0.00896731298416853), ('visually', 0.007520971819758415), ('impaired', 0.007520971819758415), ('city', 0.007231703959405422), ('using', 0.006942435633391142), ('instances', 0.006653167307376862)]<br/>



With stemmer = None parameter:<br/>
Basic Info:<br/>

| HLDAModel (current version: 0.12.7)<br/>
| 63 docs, 194213 words<br/>
| Total Vocabs: 36472, Used Vocabs: 2454<br/>
| Entropy of words: 7.26714<br/>
| Entropy of term-weighted words: 7.26714<br/>
| Removed Vocabs: people users design environment route user accessibility research navigation disabilities<br/>

Training Info:<br/>
| Iterations: 1100, Burn-in steps: 0<br/>
| Optimization Interval: 10<br/>
| Log-likelihood per word: -7.22443<br/>

Initial Parameters:<br/>
| tw: TermWeight.ONE<br/>
| min_cf: 0 (minimum collection frequency of words)<br/>
| min_df: 10 (minimum document frequency of words)<br/>
| rm_top: 10 (the number of top words to be removed)<br/>
| depth: 4 (the maximum depth level of hierarchy between 2 ~ 32767)<br/>
| alpha: [0.1] (hyperparameter of Dirichlet distribution for document-depth level, given as a single `float` in case of symmetric prior and as a list with length `depth` of `float` in case of asymmetric prior.)<br/>
| eta: 0.01 (hyperparameter of Dirichlet distribution for topic-word)<br/>
| gamma: 0.1 (concentration coeficient of Dirichlet Process)<br/>
| seed: 3948541880 (random seed)<br/>
| trained in version 0.12.7<br/>

Parameters:<br/>
| alpha (Dirichlet prior on the per-document depth level distributions)<br/>
|  [1.5013909 1.3022106 0.4230723 0.9335235]<br/>
| eta (Dirichlet prior on the per-topic word distribution)<br/>
|  0.01<br/>
| gamma (concentration coefficient of Dirichlet Process)<br/>
|  0.1<br/>
| Number of Topics: 43<br/>

Topics:<br/>
| #0 (59725, 63) : study https access public one<br/>
|   #8 (44665, 60) : used use using information based<br/>
|     #16 (4697, 11) : application system blind impaired visually<br/>
|       #24 (2655, 2) : public spaces physical wheelchair mobility<br/>
|       #25 (2006, 2) : mobility devices participants disability use<br/>
|       #38 (3922, 5) : blind surfaces participants tactile feedback<br/>
|       #66 (1842, 2) : persons planning doi disability city<br/>
|     #17 (2366, 14) : data street mobility system pedestrian<br/>
|       #27 (4559, 2) : model path cultural time proposed<br/>
|       #28 (1191, 4) : application path accessible data pedestrian<br/>
|       #35 (3595, 2) : cognitive studies neighborhood older health<br/>
|       #50 (2560, 3) : transit transportation curb pedestrian ramps<br/>
|       #56 (2634, 3) : system detection impaired visually fig<br/>
|     #18 (3822, 2) : indoor planning ieee impaired blind<br/>
|       #70 (2362, 2) : space work sample furniture survey<br/>
|     #19 (2629, 3) : travel disabled stations density transit<br/>
|       #60 (2733, 3) : transport bus walking street disability<br/>
|     #20 (1199, 3) : routing figure barriers 2021 barrier<br/>
|       #65 (3028, 3) : way participant doi 2019 indoor<br/>
|     #21 (1676, 4) : social inclusion level disability activities<br/>
|       #31 (1746, 2) : wheelchair 2019 community mobility survey<br/>
|       #69 (1394, 2) : disability 2020 housing built work<br/>
|     #22 (1784, 2) : travel level activity model selection<br/>
|       #30 (1830, 2) : confidence segment values criteria wheelchair<br/>
|     #23 (1742, 2) : care participants place doi 2019<br/>
|       #32 (2421, 2) : sensory participation experiences like children<br/>
|     #40 (681, 5) : impaired visually learning visual building<br/>
|       #63 (1030, 3) : inclusive disability students spaces accessible<br/>
|       #68 (1234, 2) : facilities public architectural data disabled<br/>
|     #42 (1667, 2) : vol impaired indoor outdoor multimodal<br/>
|       #48 (2246, 2) : health mental covid-19 anxiety depression<br/>
|     #43 (2231, 2) : elevator building simulation elevators strategy<br/>
|       #55 (1508, 2) : buildings building main accessible barriers<br/>
|     #44 (1073, 3) : participants stress spatial emotional response<br/>
|       #71 (2480, 3) : space physical model urban spaces<br/>
|     #45 (949, 4) : outcomes settings features setting may<br/>
|       #39 (1648, 2) : older adults hearing change health<br/>
|       #64 (1388, 2) : building buildings evaluation health new<br/>
|     #47 (1142, 3) : blind urban city orientation person<br/>
|       #54 (2291, 3) : planning doi open public city<br/>
|   #9 (1615, 3) : contrast visual vision elements standards<br/>
|     #41 (1997, 3) : information 2020 2021 provide blind<br/>
|       #51 (4250, 3) : impaired impairment apps paper system<br/>
|

[ 0 86 86 86] [('transport', 0.02727966383099556), ('disabled', 0.016978252679109573), ('public', 0.015261350199580193), ('people', 0.014879817143082619), ('doi', 0.012590614147484303), ('dhaka', 0.010682945139706135), ('research', 0.007630675099790096), ('accessibility', 0.007439908571541309), ('disability', 0.0064860740676522255), ('barriers', 0.006295307073742151)]<br/>
[ 0  8 17 26] [('route', 0.026824865490198135), ('risk', 0.016765540465712547), ('barriers', 0.014960020780563354), ('mobility', 0.013928295113146305), ('wheelchair', 0.012122775427997112), ('routes', 0.0116069121286273), ('https', 0.011091049760580063), ('accessibility', 0.010575187392532825), ('org', 0.009543461725115776), ('doi', 0.00928553007543087)]<br/>
[ 0 86 86 86] [('design', 0.02287166379392147), ('none', 0.010959339328110218), ('students', 0.010482845827937126), ('needs', 0.009053367190063), ('patients', 0.008259212598204613), ('satisfied', 0.008259212598204613), ('entrance', 0.007623888086527586), ('built', 0.007306226063519716), ('educational', 0.006829733029007912), ('environment', 0.006670902017503977)]<br/>
[ 0  8 16 38] [('dataset', 0.01267605647444725), ('detection', 0.01232394389808178), ('pothole', 0.011267605237662792), ('visually', 0.007746479008346796), ('impaired', 0.007042253389954567), ('sov', 0.006690140813589096), ('potholes', 0.005985915660858154), ('system', 0.005985915660858154), ('vision', 0.005633802618831396), ('lite', 0.005633802618831396)]<br/>
[ 0  8 40 57] [('health', 0.02314932644367218), ('care', 0.016026455909013748), ('sci', 0.014245739206671715), ('people', 0.01170185673981905), ('prevention', 0.011447468772530556), ('shcs', 0.011447468772530556), ('south', 0.00915797520428896), ('africa', 0.008903587237000465), ('rehabilitation', 0.008649198338389397), ('cord', 0.008394810371100903)]<br/>
[ 0  8 40 68] [('user', 0.0181590486317873), ('smart', 0.016123982146382332), ('navigation', 0.010644959285855293), ('assistant', 0.010331871919333935), ('impaired', 0.0095491549000144), ('visually', 0.009236067533493042), ('cane', 0.008453349582850933), ('people', 0.008453349582850933), ('mode', 0.00782717578113079), ('system', 0.0075140888802707195)]<br/>
[ 0  8 17 28] [('2020', 0.02282157726585865), ('museum', 0.018672199919819832), ('museums', 0.015214384533464909), ('people', 0.015214384533464909), ('doi', 0.014177040196955204), ('pandemic', 0.013485477305948734), ('2021', 0.012793914414942265), ('https', 0.01244813296943903), ('disabled', 0.008644537068903446), ('org', 0.008644537068903446)]<br/>
[ 0  8 20 65] [('accessibility', 0.023970291018486023), ('edmonton', 0.011478730477392673), ('2018', 0.01080351136624813), ('people', 0.009115462191402912), ('urban', 0.008777853101491928), ('research', 0.0077650235034525394), ('standards', 0.0077650235034525394), ('https', 0.0077650235034525394), ('canada', 0.007427413947880268), ('coe', 0.006752194371074438)]<br/>
[ 0 74 74 74] [('tactile', 0.01924114301800728), ('bus', 0.0181622002273798), ('people', 0.0163639634847641), ('surfaces', 0.010789426043629646), ('design', 0.008631541393697262), ('stops', 0.008271893486380577), ('stop', 0.007912245579063892), ('visual', 0.0071929506957530975), ('universal', 0.006114008370786905), ('figure', 0.0055745369754731655)]<br/>
[ 0  0  8 47] [('disability', 0.017389917746186256), ('social', 0.015315890312194824), ('facilities', 0.011805999092757702), ('students', 0.011486917734146118), ('buildings', 0.01100829616189003), ('pwds', 0.010529674589633942), ('education', 0.009093809872865677), ('disabilities', 0.007817485369741917), ('university', 0.007657945156097412), ('built', 0.00749840447679162)]<br/>
[ 0  8 20 65] [('julian', 0.06896551698446274), ('dresden', 0.06896551698446274), ('jan', 0.06896551698446274), ('striegl', 0.06896551698446274), ('accessible', 0.03448275849223137), ('weber', 0.03448275849223137), ('informatik', 0.03448275849223137), ('fabian', 0.03448275849223137), ('ders', 0.03448275849223137), ('lu¨', 0.03448275849223137)]<br/>
[ 0 86 86 86] [('information', 0.011760243214666843), ('data', 0.009484066627919674), ('user', 0.00834597833454609), ('doi', 0.0075872535817325115), ('network', 0.007397572044283152), ('directions', 0.007018209435045719), ('wayfinding', 0.006828527897596359), ('vol', 0.006828527897596359), ('persons', 0.006259484216570854), ('navigation', 0.005690440069884062)]<br/>
[ 0 87 87 87] [('system', 0.025241902098059654), ('model', 0.0092553636059165), ('object', 0.008834665641188622), ('signs', 0.008413966745138168), ('proposed', 0.007572570350021124), ('test', 0.007572570350021124), ('participants', 0.007572570350021124), ('detection', 0.007151871919631958), ('easy', 0.007151871919631958), ('user', 0.0067311739549040794)]<br/>
[0 7 7 7] [('lane', 0.021570103242993355), ('detection', 0.014547279104590416), ('road', 0.012916980311274529), ('methods', 0.01266616489738226), ('segmentation', 0.009781790897250175), ('method', 0.009280160069465637), ('pedestrian', 0.00827690027654171), ('network', 0.00802608486264944), ('color', 0.007900677621364594), ('image', 0.007148231845349073)]<br/>
[ 0 77 77 77] [('2010', 0.017247987911105156), ('individuals', 0.015139900147914886), ('behaviours', 0.013031813316047192), ('2009', 0.011498658917844296), ('autism', 0.011307014152407646), ('challenge', 0.011115370318293571), ('architecture', 0.010348792187869549), ('mostafa', 0.00977385975420475), ('brand', 0.0095822149887681), ('rca', 0.009007282555103302)]<br/>
[ 0 84 84 84] [('learning', 0.018307767808437347), ('environment', 0.015338941477239132), ('universities', 0.013854527845978737), ('design', 0.011380504816770554), ('education', 0.011380504816770554), ('university', 0.011380504816770554), ('inclusion', 0.010390895418822765), ('universal', 0.009896091185510159), ('0-25', 0.009896091185510159), ('students', 0.009896091185510159)]<br/>
[ 0  8 17 56] [('road-traffic', 0.01535735372453928), ('mortality', 0.014766686595976353), ('cities', 0.013388462364673615), ('deaths', 0.012994684278964996), ('higher', 0.011616460047662258), ('urban', 0.010435124859213829), ('population', 0.010041346773505211), ('traffic', 0.009647568687796593), ('city', 0.009647568687796593), ('density', 0.008466233499348164)]<br/>
[ 0  8 17 56] [('impaired', 0.008439566008746624), ('visually', 0.008439566008746624), ('user', 0.007790368050336838), ('using', 0.0063739377073943615), ('data', 0.006314919795840979), ('system', 0.006196883972734213), ('learning', 0.005960811860859394), ('obstacle', 0.005960811860859394), ('sensors', 0.005606704391539097), ('used', 0.005488668568432331)]<br/>
[ 0  8 40 63] [('children', 0.032258063554763794), ('school', 0.019825268536806107), ('schools', 0.018145160749554634), ('inclusive', 0.01646505296230316), ('disability', 0.013776881620287895), ('special', 0.011424731463193893), ('ghana', 0.010416666977107525), ('education', 0.010080644860863686), ('needs', 0.010080644860863686), ('disabilities', 0.009072580374777317)]<br/>
[ 0  8 16 66] [('dataset', 0.025744864717125893), ('images', 0.013884871266782284), ('datasets', 0.010702922008931637), ('object', 0.01041365321725607), ('detection', 0.00896731298416853), ('visually', 0.007520971819758415), ('impaired', 0.007520971819758415), ('city', 0.007231703959405422), ('using', 0.006942435633391142), ('instances', 0.006653167307376862)]<br/>

In [ ]:
HLDAModel